In [1]:
!pip install kaggle

In [12]:
!pip install opencv-python

In [15]:
! pip install -q kaggle

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf

from tqdm.auto import tqdm

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, Flatten, MaxPool2D, Activation
from keras.utils import to_categorical
from IPython.display import SVG
from tensorflow.keras.utils import image_dataset_from_directory 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img 
from tensorflow.keras.preprocessing import image_dataset_from_directory 

import os
import cv2
import shutil
import matplotlib.image as mpimg

In [8]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
!kaggle datasets download - d salader/dogs-vs-cats

Traceback (most recent call last):
  File "C:\Users\asus\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\asus\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\asus\anaconda3\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "C:\Users\asus\anaconda3\lib\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "C:\Users\asus\anaconda3\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\asus\.kaggle. Or use the environment method.


In [5]:
from zipfile import ZipFile 
  
data_path = "C:/Users/asus/Downloads/Dogs-vs-Cats.zip"
  
with ZipFile(data_path, 'r') as zip: 
    zip.extractall("C:/Users/asus/Downloads/Dogs-vs-Cats") 
    print('The data set has been extracted.') 

The data set has been extracted.


# Data Preperation for TRaining 

In [6]:
base_dir = "C:/Users/asus/Downloads/Dogs-vs-Cats"
  
# Create datasets 
train_datagen = image_dataset_from_directory(base_dir, 
                                                  image_size=(256,256), 
                                                  subset='training', 
                                                  seed = 1, 
                                                 validation_split=0.1, 
                                                  batch_size= 32) 
test_datagen = image_dataset_from_directory(base_dir, 
                                                  image_size=(256,256), 
                                                  subset='validation', 
                                                  seed = 1, 
                                                 validation_split=0.1, 
                                                  batch_size= 32)

Found 37500 files belonging to 2 classes.
Using 33750 files for training.
Found 37500 files belonging to 2 classes.
Using 3750 files for validation.


In [7]:
## Normalizing 

def process(image, label):
    image= tf.cast(image/255, tf.float32)
    return image, label

train_datagen = train_datagen.map(process)
test_datagen = test_datagen.map(process)

# Model Building

In [8]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding="valid", activation='relu', input_shape=(256,256,3)))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding="valid", activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding="valid", activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

# Model Compilation

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Training

In [ ]:
history = model.fit(train_datagen, epochs=10, validation_data = test_datagen)

Epoch 1/10
1055/1055 [==============================] - 1864s 2s/step - loss: 0.6390 - accuracy: 0.6667 - val_loss: 0.6381 - val_accuracy: 0.6661
Epoch 2/10
1055/1055 [==============================] - 1769s 2s/step - loss: 0.6372 - accuracy: 0.6667 - val_loss: 0.6367 - val_accuracy: 0.6661
Epoch 3/10
1055/1055 [==============================] - 1478s 1s/step - loss: 0.6371 - accuracy: 0.6667 - val_loss: 0.6370 - val_accuracy: 0.6661
Epoch 4/10
 254/1055 [======>.......................] - ETA: 18:04 - loss: 0.6396 - accuracy: 0.6634

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='train')
plt.plot(history.history['val_accuracy'], color='red', label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], color='red', label='train')
plt.plot(history.history['val_loss'], color='red', label='validation')
plt.legend()
plt.show()

# Model Testing and Prediction

In [ ]:
from keras.preprocessing import image 
  
#Input image 
test_image = image.load_img("C:/Users/asus/Downloads/cat image.avif",target_size=(256,256)) 
  
#For show image 
plt.imshow(test_image) 
test_image = image.img_to_array(test_image) 
test_image = np.expand_dims(test_image,axis=0) 
  
# Result array 
result = model.predict(test_image) 
  
#Mapping result array with the main name list 
i=0
if(result>=0.5): 
    print("Dog") 
else: 
    print("Cat")